In [1]:
%matplotlib qt

In [21]:
import matplotlib.pyplot as plt
import numpy as np
from prettyplots import pretty_axis
import pickle
from astropy.table import Table
import astropy.units as u
import os
from matplotlib.gridspec import GridSpec
import json

# Plot stellar SEDs

In [3]:
alf_cen_infile = {
    'peak':'../atm/stellar_flux/alf_cen_peak.txt',
    'min':'../atm/stellar_flux/alf_cen_min.txt',
}

solar_infile = {
    'gueymard':'../atm/stellar_flux/gueymard_solar.txt',
    'woods':'../atm/stellar_flux/woods_solar.txt',
}


alf_cen_data = {
    'peak':Table.read(alf_cen_infile['peak'],format='ascii'),
    'min':Table.read(alf_cen_infile['min'],format='ascii'),
}

solar_data = {
    'gueymard':Table.read(solar_infile['gueymard'],format='ascii',names=['wavelength','flux']),
    'woods':Table.read(solar_infile['woods'],format='ascii'),
}


In [19]:
fig,ax = plt.subplots(1,1,figsize=(9,4),layout='constrained')
pretty_axis(ax)



#ax.loglog(solar_data['gueymard']['wavelength'],solar_data['gueymard']['flux'],label='Gueymard solar')
ax.loglog(alf_cen_data['peak']['wavelength']*10,alf_cen_data['peak']['flux'],label=r'$\alpha$ Cen peak',zorder=4)
ax.loglog(alf_cen_data['min']['wavelength']*10,alf_cen_data['min']['flux'],label=r'$\alpha$ Cen min',zorder=4)
ax.loglog(solar_data['woods']['wavelength']*10,solar_data['woods']['flux'],label='Sun',zorder=2)
ax.legend(fontsize=14)
ax.set_xlabel(r'Wavelength [$\rm{\AA}$]',fontsize=18)
ax.set_ylabel(r'Flux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]',fontsize=18)

Text(0, 0.5, 'Flux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]')

In [20]:
plt.savefig('../paper-figures/stellar_seds.png',format='png')
plt.savefig('../paper-figures/stellar_seds.pdf',format='pdf')

# Plot atmosphere retrievals

In [13]:
with open('./label_locs.json') as file:
    label_locs = json.load(file)

## Terrestrial

In [7]:
min_file = '../output/terrestrial/alf_cen_min.vul'
peak_file = '../output/terrestrial/alf_cen_peak.vul'
earth_file = '../output/Earth.vul'

In [8]:
specs_to_plot = ['H','H2','H2O','CO','CO2','CH4','N2','O','O2']

fig = plt.figure(layout='constrained',figsize=(12,6))

gs = GridSpec(1,3,figure=fig)
ax = [
    fig.add_subplot(gs[0:2]),
    fig.add_subplot(gs[2])
]

for subax in ax:
    pretty_axis(subax)

for infile,linestyle,label in zip([peak_file,min_file,earth_file],['-','--',':'],['peak','min','Earth']):
    with open(infile,'rb') as handle:
        data = pickle.load(handle)
        pressure = data['atm']['pco']/1.e6
        
    mixing_ratios = {}
    
    for species in specs_to_plot:
        ind = data['variable']['species'].index(species)
        mixing_ratios[species] = data['variable']['ymix'][:,ind]

    i = 0
    for sp,mr in mixing_ratios.items():
        for char in sp:
            if char.isdigit():
                # Format species for label
                sp = sp.replace(char,"$_%s$"%(char))
        if 'peak' in infile:
            ax[0].plot(mr,pressure,label=sp,ls=linestyle,c='C%s'%(i))
        else:
            ax[0].plot(mr,pressure,ls=linestyle,c='C%s'%(i))
        i += 1

ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].invert_yaxis() 
ax[0].minorticks_on()
ax[0].set_ylim((data['atm']['pco'][0]/1e6,data['atm']['pco'][-1]/1e6))
ax[0].set_ylabel("Pressure [bar]",fontsize=20)
ax[0].set_xlabel("Mixing ratio",fontsize=20)
ax[0].legend(fontsize=14)

'''
# Plot SEDs
ax[1].loglog(solar_data['woods']['wavelength'],solar_data['woods']['flux'],label='Woods solar')
ax[1].loglog(alf_cen_data['peak']['wavelength'],alf_cen_data['peak']['flux'],label=r'$\alpha$ Cen peak')
ax[1].loglog(alf_cen_data['min']['wavelength'],alf_cen_data['min']['flux'],label=r'$\alpha$ Cen min')
ax[1].legend(fontsize=12)
ax[1].set_xlabel(r'Wavelength [nm]',fontsize=18)
ax[1].set_ylabel(r'Stellar lux [erg s$^{-1}$ cm$^{-2}$ nm$^{-1}$]',fontsize=18)
'''

# Plot TP profile
ax[1].semilogy(data['atm']['Tco'], pressure)
ax[1].set_xlabel('Temperature [K]',fontsize=18)
ax[1].set_ylabel('Pressure [bar]',fontsize=18)
ax[1].invert_yaxis()
ax[1].sharey(ax[0])

ax[0].plot([1e-12,1e-10],[1e-1,1e-1],ls='-',c='k')
ax[0].text(s='Peak',x=1.5e-10,y=1e-1,fontsize=14,verticalalignment='center')

ax[0].plot([1e-12,1e-10],[2e-1,2e-1],ls='--',c='k')
ax[0].text(s='Min',x=1.5e-10,y=2e-1,fontsize=14,verticalalignment='center')

ax[0].plot([1e-12,1e-10],[4e-1,4e-1],ls=':',c='k')
ax[0].text(s='Earth',x=1.5e-10,y=4e-1,fontsize=14,verticalalignment='center')

Text(1.5e-10, 0.4, 'Earth')

In [9]:
plt.savefig('../paper-figures/atm_terrestrial.png',format='png')
plt.savefig('../paper-figures/atm_terrestrial.pdf',format='pdf')

## Gas giant

In [10]:
infiles = {
    'min':'../output/gas-giant/alf_cen_min.vul',
    'peak':'../output/gas-giant/alf_cen_peak.vul'
}

In [16]:
specs_to_plot = ['H','He','H2','H2O','CO','CO2','CH4','N2','O','O2']

fig = plt.figure(layout='constrained',figsize=(12,6))

gs = GridSpec(1,3,figure=fig)

ax = [
    fig.add_subplot(gs[0:2]),
    fig.add_subplot(gs[2])
]

for subax in ax:
    pretty_axis(subax)

for profile,ls in zip(['min','peak'],['--','-']):
    with open(infiles[profile],'rb') as handle:
        data = pickle.load(handle)
        pressure = data['atm']['pco']/1.e6
        
    mixing_ratios = {}
    
    for species in specs_to_plot:
        ind = data['variable']['species'].index(species)
        mixing_ratios[species] = data['variable']['ymix'][:,ind]

    i = 0
    for sp,mr in mixing_ratios.items():
        for char in sp:
            if char.isdigit():
                # Format species for label
                sp = sp.replace(char,"$_%s$"%(char))
        if profile=='peak':
            ax[0].plot(mr,pressure,label=sp,ls=ls,c='C%s'%(i))
        else:
            ax[0].plot(mr,pressure,ls=ls,c='C%s'%(i))
        i += 1

for i,(species,loc) in enumerate(label_locs['gas'].items()):
    if species in ['H','O$_2$']:
        ax[0].text(s=species,x=loc[0],y=loc[1],
                   color=f"C{i}",
                   fontsize=14,
                   verticalalignment='center')

ax[0].set_yscale('log')
ax[0].set_xscale('log')
ax[0].invert_yaxis() 
ax[0].minorticks_on()
ax[0].set_ylim((data['atm']['pco'][0]/1e6,data['atm']['pco'][-1]/1e6))
ax[0].set_ylabel("Pressure [bar]",fontsize=20)
ax[0].set_xlabel("Mixing ratio",fontsize=20)
ax[0].legend(fontsize=14)

# Plot TP profile
ax[1].semilogy(data['atm']['Tco'], pressure)
ax[1].set_xlabel('Temperature [K]',fontsize=18)
ax[1].set_ylabel('Pressure [bar]',fontsize=18)
ax[1].invert_yaxis()
ax[1].sharey(ax[0])

ax[0].plot([1e-17,1e-15],[1e-1,1e-1],ls='-',c='k')
ax[0].text(s='Peak',x=3e-15,y=1e-1,fontsize=14,verticalalignment='center')

ax[0].plot([1e-17,1e-15],[2e-1,2e-1],ls='--',c='k')
ax[0].text(s='Min',x=3e-15,y=2e-1,fontsize=14,verticalalignment='center')

Text(3e-15, 0.2, 'Min')

In [18]:
plt.savefig('../paper-figures/atm_gas-giant.png',format='png')
plt.savefig('../paper-figures/atm_gas-giant.pdf',format='pdf')